In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook
%matplotlib inline

In [2]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.calculation.job import JobCalculation

from plot_methods.plot_fleur_aiida import plot_spectra
import plot_methods

from base64 import b64encode
import StringIO
import numpy as np
import ipywidgets as ipw
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from ase.data import covalent_radii, atomic_numbers
from ase.data.colors import cpk_colors
from ase.neighborlist import NeighborList

# Search Criteria

In [3]:
############################   START OF PREPROCESSING   ###############################

In [4]:
PREPROCESS_VERSION = 6.004

def preprocess_newbies():
    qb = QueryBuilder()
    qb.append(WorkCalculation, filters={
        'attributes._process_label': 'fleur_initial_cls_wc',
        'or':[
               {'extras': {'!has_key': 'preprocess_version'}},
               {'extras.preprocess_version': {'<': PREPROCESS_VERSION}},
           ],
    })
    
    
    for m in qb.all(): # iterall() would interfere with set_extra()
        n = m[0]
        if not n.is_sealed:
            print("Skipping underway workchain PK %d"%n.pk)
            continue
        if 'obsolete' not in n.get_extras():
            n.set_extra('obsolete', False)
        try:
            preprocess_one(n)
            n.set_extra('preprocess_successful', True)
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            print("Preprocessed PK %d"%n.pk)
        except Exception as e:
            n.set_extra('preprocess_successful', False)
            n.set_extra('preprocess_error', str(e))
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            print("Failed to preprocess PK %d: %s"%(n.pk, e))

In [5]:
def preprocess_one(workcalc):
    # TODO : do we really want to set all the extra keys?
    def get_calc_by_label(workcalc, label):
        qb = QueryBuilder()
        qb.append(WorkCalculation, filters={'uuid':workcalc.uuid})
        qb.append(JobCalculation, output_of=WorkCalculation, filters={'label':label})
        # process label?
        if qb.count() != 1:
            raise(Exception("Could not find %s calculation."%label))
        calc = qb.first()[0]
        return calc
  
    # Formula
    structure = workcalc.inp.structure
    ase_struct = structure.get_ase()
    formula = ase_struct.get_chemical_formula()
    symmetry = ''
    workcalc.set_extra('formula', formula)
    workcalc.set_extra('structure_description', structure.description)
    
    
    # thumbnail
    thumbnail = render_thumbnail(ase_struct)
    workcalc.set_extra('thumbnail', thumbnail)
    
    # ensure all steps succeed
    #all_steps = ['fleur_scf_wc']

    #for label in all_steps:
    #    calc = get_calc_by_label(workcalc, label)
    #    if calc.get_state() != 'FINISHED':
    #        raise(Exception("Calculation %s in state %s."%(label, calc.get_state())))
    #    if "aiida.out" not in calc.out.retrieved.get_folder_list():
    #        raise(Exception("Calculation %s did not retrive aiida.out"%label))
    #    #fn = calc.out.retrieved.get_abs_path("aiida.out")
    #    #content = open(fn).read()
    #    #if "JOB DONE." not in content:
    #    #    raise(Exception("Calculation %s did not print JOB DONE."%label))
    
    # energies
    #scf_calc = get_calc_by_label(workcalc, "scf")
    #assert scf_calc.res['fermi_energy_units'] == 'eV'
    #fermi_energy = scf_calc.res['fermi_energy']
    #assert scf_calc.res['energy_units'] == 'eV'
    #workcalc.set_extra('total_energy', scf_calc.res['energy'])
    #workcalc.set_extra('opt_structure_uuid', scf_calc.inp.structure.uuid)
    
    # outputnode
    #wc_outputpara_dict = workcalc.out.output_parameters
    
    # gap
    #bandgap
    #workcalc.set_extra('gap', gap)
        
    # vacuum level
    #export_hartree_calc = get_calc_by_label(workcalc, "export_hartree")
    #fn = export_hartree_calc.out.retrieved.get_abs_path("vacuum_hartree.dat")
    #data = np.loadtxt(fn)
    #vacuum_level = np.mean(data[:,2]) * 27.211385 * 0.5
    #workcalc.set_extra('vacuum_level', vacuum_level)
    
    # store shifted energies
    #workcalc.set_extra('fermi_energy', fermi_energy - vacuum_level)
    #if is_insulator:
    #    workcalc.set_extra('homo', homo - vacuum_level)
    #    workcalc.set_extra('lumo', lumo - vacuum_level)
    #else:
    #    workcalc.set_extra('homo', fermi_energy - vacuum_level)
    #    workcalc.set_extra('lumo', fermi_energy - vacuum_level)

In [6]:
def extract_information(workcalc):
    from aiida_fleur.tools.StructureData_util import get_spacegroup
    return_dict = {}
    
    # structure
    structure = workcalc.inp.structure
    structure_uuid = structure.uuid
    formula = structure.get_formula()
    space_group = get_spacegroup(structure)
    return_dict['space_group'] = space_group
    return_dict['structure_uuid'] = structure_uuid
    
    # outputnode
    wc_outputpara_dict = workcalc.out.output_inital_cls_wc_para.get_dict()
    
    #bandgap
    gap = wc_outputpara_dict.get('bandgap', {})
    return_dict['bandgap'] = gap.get(formula, -100) # currently this is a dict...    
    
    #efermi
    fermi_energy = wc_outputpara_dict.get('fermi_energy', {})
    return_dict['fermi_energy'] = fermi_energy.get(formula, -100)
    
    return return_dict

In [7]:
from aiida_fleur.tools.StructureData_util import get_spacegroup


get_spacegroup()

TypeError: get_spacegroup() takes exactly 1 argument (0 given)

In [ ]:
def render_thumbnail(ase_struct):
    s = ase_struct.repeat((2,1,1))
    cov_radii = [covalent_radii[a.number] for a in s]
    nl = NeighborList(cov_radii, bothways = True, self_interaction = False)
    nl.update(s)
    
    fig, ax = plt.subplots()
    ax.set_aspect(1)
    ax.axes.set_xlim([0,s.cell[0][0]])
    ax.axes.set_ylim([5,s.cell[1][1]-5])
    #ax.set_axis_bgcolor((0.423,0.690,0.933))
    ax.set_axis_bgcolor((0.85,0.85,0.85))
    ax.axes.get_yaxis().set_visible(False)

    #name = ase_struct.get_chemical_formula() # get name before repeat
    #ax.set_xlabel(name, fontsize=12)
    ax.tick_params(axis='x', which='both', bottom='off', top='off',labelbottom='off')
    
    for at in s:
        #circles
        x,y,z = at.position
        n = atomic_numbers[at.symbol]
        ax.add_artist(plt.Circle((x,y), covalent_radii[n]*0.5, color=cpk_colors[n], fill=True, clip_on=True))
        #bonds
        nlist = nl.get_neighbors(at.index)[0]
        for theneig in nlist:
            x,y,z = (s[theneig].position +  at.position)/2
            x0,y0,z0 = at.position
            if (x-x0)**2 + (y-y0)**2 < 2 :
                ax.plot([x0,x],[y0,y],color=cpk_colors[n],linewidth=2,linestyle='-')

    img = StringIO.StringIO()
    fig.savefig(img, format="png", dpi=72, bbox_inches='tight')
    return b64encode(img.getvalue())

In [8]:
############################   END OF PREPROCESSING   ###############################

In [9]:
def search():

    results.value = "preprocessing..."
    preprocess_newbies()
    
    results.value = "searching..."
    
    # html table header
    html  = '<style>#aiida_results td,th {padding: 2px}</style>' 
    #html += '<form action="compare.ipynb" method="get" target="_blank">'
    html += '<table border=1 id="aiida_results" style="margin:10px;"><tr>'
    html += '<th></th>'
    html += '<th>PK</th>'
    #html += '<th>UUID</th>'
    html += '<th>Workchain type</th>'
    html += '<th>Creation Time</th>'
    html += '<th>Formula</th>'
    html += '<th>Symmetry</th>'
    html += '<th>GAP [eV]</th>'
    html += '<th>Fermi Energy [eV]</th>'
    html += '<th>Structure</th>'
    html += '<th>Spectrum preview</th>'
    html += '<th></th>'
    html += '</tr>'

    # query AiiDA database
    filters = {}
    filters['attributes._process_label'] = 'fleur_initial_cls_wc'
    #filters['extras.preprocess_version'] = PREPROCESS_VERSION
    #filters['extras.preprocess_successful'] = True
    #filters['extras.obsolete'] = False
    
    pk_list = inp_pks.value.strip().split()
    if pk_list:
        # TODO make a pk and uuid list
        filters['id'] = {'in': pk_list}
        #filters['uuid] = {'in': uuid_list}

    formula_list = inp_formula.value.strip().split()
    if inp_formula.value:
        # TODO get formula rather from structure
        # or workchain node itself
        filters['extras.formula'] = {'in': formula_list}

    def add_range_filter(bounds, label):
        filters['extras.'+label] = {'and':[{'>=':bounds[0]}, {'<':bounds[1]}]}

    #add_range_filter(inp_gap.value, "gap")
    #add_range_filter(inp_efermi.value, "fermi_energy")

    
    qb = QueryBuilder()        
    qb.append(WorkCalculation, filters=filters)
    qb.order_by({WorkCalculation:{'ctime':'desc'}})
    qb.limit(10)
    
    for i, node_tuple in enumerate(qb.iterall()):
        node = node_tuple[0]
        extras = node.get_extras()
        thumbnail = extras.get('thumbnail')
        description = extras.get('structure_description', '')
        structure_uuid = extras.get('structure_uuid')
        #opt_structure_uuid = node.get_extra('opt_structure_uuid')
        print node
        print type(node)
        # TODO
        res_node = extract_information(node)
        # append table row
        html += '<tr>'
        html += '<td><input type="checkbox" name="pk" value="%s"></td>'%node.pk
        html += '<td><a target="_blank" href="../../aiida/aiida_graph_browser.ipynb?pk=%d">%d</a></td>' % (node.pk, node.pk)
        #html += '<td>%s</td>' % node.uuid
        html += '<td>%s</td>' % ''
        html += '<td>%s</td>' % node.ctime.strftime("%Y-%m-%d %H:%M")
        html += '<td>%s</td>' % node.get_extra('formula', '..')
        html += '<td>%s</td>' % node.get_extra('space_group' , '..')
        html += '<td>%f</td>' % res_node.get('bandgap', -100)
        html += '<td>%f</td>' % res_node.get('fermi_energy', -100)
        #html += '<td></td>'
        #html += '<td></td>'
        #html += '<td></td>'
        #html += '<td></td>'
        html += '<td></td>'
        #html += '<td></td>'
        #html += '<td></td>'
        html += '<td><a target="_blank" href="./export_structure.ipynb?uuid=%s">'%structure_uuid
        html += '<img src="data:image/png;base64,%s" title="%s"></a></td>' % (thumbnail, description)
        html += '<td></td>'
        #html += '<td><a target="_blank" href="./show.ipynb?pk=%s">Show</a><br>'%node.pk
        #html += '<a target="_blank" href="./show_pdos.ipynb?pk=%s">PDOS</a></td>'%node.pk
        html += '</tr>'

    html += '</table>'
    html += 'Found %d matching entries.<br>'%qb.count()
    html += '<input type="submit" value="Compare">'
    html += '</form>'

    results.value = html

In [10]:
# search UI
style = {"description_width":"100px"}
layout = ipw.Layout(width="692px")#ipw.Layout(width="592px")
inp_plugin = ipw.Text(description='Plugins:', placeholder='e.g. fleur.fleur (space separated)', layout=layout, style=style)
inp_codes = ipw.Text(description='Codes:', placeholder='e.g. fleur_0.27@localhost (space separated)', layout=layout, style=style)
inp_wc_types = ipw.Text(description='Workchain types:', placeholder='e.g. fleur_initial_cls_wc (space separated, default)', layout=layout, style=style)
inp_pks = ipw.Text(description='PKs:', placeholder='e.g. 4062 4753 (space separated)', layout=layout, style=style)
inp_formula = ipw.Text(description='Formulas:', placeholder='e.g. Be2W Be12W', layout=layout, style=style)
inp_elements = ipw.Text(description='Contains Elements:', placeholder='e.g. Be W (space separated)', layout=layout, style=style)
inp_cls = ipw.Text(description='Core-levels:', placeholder='e.g. Be1s W4f (space separated)', layout=layout, style=style)
inp_max = ipw.Text(description='Maximum results:', placeholder='e.g. 10 (default 100)', layout=layout, style=style)


def slider(desc, min, max):
    return ipw.FloatRangeSlider(description=desc, min=min, max=max, 
                                    value=[min, max], step=0.05, layout=layout, style=style)

inp_gap = slider("Gap [eV]:", 0.0, +3.0)
inp_efermi = slider("Fermi Energy [eV]:", -6.0, -1.0)
search_crit = [inp_plugin, inp_codes, inp_wc_types, inp_pks, inp_formula, inp_elements, inp_cls, inp_max, inp_gap, inp_efermi]

In [11]:
def on_click(b):
    with info_out:
        clear_output()
        search()

button = ipw.Button(description="Search")
button.on_click(on_click)
results = ipw.HTML()
info_out = ipw.Output()
app = ipw.VBox(children=search_crit + [button, results, info_out])
display(app)

VBox(children=(Text(value=u'', description=u'Plugins:', layout=Layout(width=u'692px'), placeholder=u'e.g. fleur.fleur (space separated)', style=DescriptionStyle(description_width=u'100px')), Text(value=u'', description=u'Codes:', layout=Layout(width=u'692px'), placeholder=u'e.g. fleur_0.27@localhost (space separated)', style=DescriptionStyle(description_width=u'100px')), Text(value=u'', description=u'Workchain types:', layout=Layout(width=u'692px'), placeholder=u'e.g. fleur_initial_cls_wc (space separated, default)', style=DescriptionStyle(description_width=u'100px')), Text(value=u'', description=u'PKs:', layout=Layout(width=u'692px'), placeholder=u'e.g. 4062 4753 (space separated)', style=DescriptionStyle(description_width=u'100px')), Text(value=u'', description=u'Formulas:', layout=Layout(width=u'692px'), placeholder=u'e.g. Be2W Be12W', style=DescriptionStyle(description_width=u'100px')), Text(value=u'', description=u'Contains Elements:', layout=Layout(width=u'692px'), placeholder=u'e.g. Be W (space separated)', style=DescriptionStyle(description_width=u'100px')), Text(value=u'', description=u'Core-levels:', layout=Layout(width=u'692px'), placeholder=u'e.g. Be1s W4f (space separated)', style=DescriptionStyle(description_width=u'100px')), Text(value=u'', description=u'Maximum results:', layout=Layout(width=u'692px'), placeholder=u'e.g. 10 (default 100)', style=DescriptionStyle(description_width=u'100px')), FloatRangeSlider(value=(0.0, 3.0), description=u'Gap [eV]:', layout=Layout(width=u'692px'), max=3.0, step=0.05, style=SliderStyle(description_width=u'100px')), FloatRangeSlider(value=(-6.0, -1.0), description=u'Fermi Energy [eV]:', layout=Layout(width=u'692px'), max=-1.0, min=-6.0, step=0.05, style=SliderStyle(description_width=u'100px')), Button(description=u'Search', style=ButtonStyle()), HTML(value=u''), Output()))

# Search Results

# All workchain nodes

collect here all successfull initial_cls and corehole wc on fusion materials

## Pure Elements (for filling)

In [ ]:
# Be
# W
W_wc_uuids = ['f8b12b23-0b71-45a1-9040-b51ccf379439']
# Ti
# Ta

## Be-Ti

In [ ]:
all_wc_BeTi_uuid = ['107b0727-15cf-4436-b614-79801cdadd8c', '0f25075d-6a7f-48a1-b837-c419eafc3017',
                    '0df9be35-5cb8-482f-b3f0-3f38531e8983', '474a6902-bbd6-4bdb-b324-aab0932f85e6',
                    'dff56acf-59d4-4056-80dc-90d76917fa78', '8e2896bf-0e2a-4a96-b494-11bf34e790e7', 
                    'd50bb22c-a3e5-4837-9d26-edc8b020a8f8']
Be_ti_list_dict = [
    {'spacegroup': u'Pm-3m (221)', 'uuid': u'3d53b191-0753-4bbb-bb48-e3b196ef87bd', 
     'structure': 'BeTi'}, 
    {'spacegroup': u'Fd-3m (227)', 'uuid': u'34903c2b-f152-4f3c-a729-38bab479f199', 
     'structure': 'Be16Ti8'}, 
    {'spacegroup': u'R-3m (166)', 'uuid': u'37e83f94-be97-426d-9831-ecc98db9abe0', 
     'structure': 'Be27Ti9'}, 
    {'spacegroup': u'Cmcm (63)', 'uuid': u'e5da7184-cadc-4af0-b748-7fe037c88a1c', 
     'structure': 'Be34Ti4'}, 
    {'spacegroup': u'R-3m (166)', 'uuid': u'f08180a4-62a2-455f-9223-465d56acc4c3', 
     'structure': 'Be51Ti6'}, 
    #{'spacegroup': u'Amm2 (38)', 'uuid': u'd2c0809e-b9ce-4e3a-bd2a-e7d080d4cf78', 'structure': 'Be12Ti'}, 
    {'spacegroup': u'Amm2 (38)', 'uuid': u'7c9a87c1-5f86-480a-9108-5cfc2a5e8991', 
     'structure': 'Be12Ti'}, 
    #{'spacegroup': u'I4/mmm (139)', 'uuid': u'da216c60-adf0-4e83-9557-8f9405976dc6', 'structure': 'Be24Ti2'}, 
    {'spacegroup': u'I4/mmm (139)', 'uuid': u'a1968b7e-76d7-46bc-81d7-de26683f563a', 
     'structure': 'Be24Ti2'}]
all_be_ti_structures_uuid_refine = [u'711da5b2-5b09-4892-82af-79d783ccffa4', u'51b53332-12f2-4014-8c5b-0e3669369d49', 
                                    u'a7d7eb07-6fb9-449d-8b62-a8d5e9e437d3', u'db93ef57-659f-499e-a481-32a7182a316e', 
                                    u'4bda0721-9202-4b1e-ae35-53d59d63d153', u'f6cf556d-ad68-40ae-8ece-006b861c0a3a', 
                                    u'660cd37b-67ca-47df-a622-236ef227ace1']

## Be-Ta

In [ ]:
BeTa_initial_state_wcs = ['37f99e98-c375-45fa-95b1-53eb80a5bfd9',
                          '8fc9dd6c-1f66-4c76-904f-703b81614999',
                          '9d4a3111-05e3-4572-a53a-d7799419fa38',
                          '48f2e31b-529c-43a1-a6d2-8c06d6a1c5b3',
                          'a6dbca9d-b9fa-442a-aff5-58f667c8a98f',
                          '54996657-4378-4172-b21d-9f782268d993']
# one base struc was already primitive
be_ta_refine_dis = [{'spacegroup': u'I4/mmm (139)', 'uuid': u'29d13b3d-3b43-4d8e-82ff-dc50cbbc26f5', 
                     'structure': 'Be12Ta'}, 
                    {'spacegroup': u'R-3m (166)', 'uuid': u'576cadf2-cc6f-48d9-bb9b-7cff33172b85', 
                     'structure': 'Be17Ta2'}, 
                    {'spacegroup': u'R-3m (166)', 'uuid': u'01bcc76f-2556-4ae6-a268-63f8e6f1e8d1', 
                     'structure': 'Be9Ta3'}, 
                    {'spacegroup': u'Fd-3m (227)', 'uuid': u'eef5e075-d4ce-423d-892d-59d2b2cfd87b', 
                     'structure': 'Be4Ta2'}, 
                    {'spacegroup': u'P4/mbm (127)', 'uuid': u'afdd35ff-5e63-4b6b-9991-f7ae8290497e', 
                     'structure': 'Be4Ta6'}, 
                    {'spacegroup': u'I4/mcm (140)', 'uuid': u'27f0d999-9ea2-4453-a96b-3f267c0bafe2', 
                     'structure': 'Be2Ta4'}]

## Be-W

In [ ]:
bew_init_wc_uuids = ['1e32880a-bdc9-4081-a5da-be04860aa1bc', 
                     '4f685bc5-b5fb-46d3-aad6-e0f512c3313d', 
                     '045d3071-f442-46b4-8d6b-3c85d72b24d4']

w_be_refine_dis = [{'spacegroup': u'Fd-3m (227)', 'uuid': u'5cc04e73-7058-4a3e-b8da-6a385b4e7050', 
                    'structure': 'Be44W2'}, 
                   {'spacegroup': u'I4/mmm (139)', 'uuid': u'0a225156-25e1-49ef-9ce8-d67e20b42725', 
                    'structure': 'Be12W'}, 
                   {'spacegroup': u'Cmcm (63)', 'uuid': u'cd5a586c-9dd2-4971-94bc-2ff62666e86b', 
                    'structure': 'Be8W4'}]

# Visualization

## W

In [ ]:
# Plot all W 4f spectra and their components
for uuid in W_wc_uuids:
    plot_spectra(uuid, energy_range=[30.0,36], energy_grid=0.02)

## Be-Ti

In [ ]:
# Plot all Be 1s spectra and their components
for uuid in all_wc_BeTi_uuid:
    plot_spectra(uuid, energy_range=[109.5,112.2], energy_grid=0.02)

## Be-Ta

In [ ]:
# Plot all Be 1s spectra and their components
for uuid in BeTa_initial_state_wcs:
    plot_spectra(uuid, energy_range=[108.5,112.2], energy_grid=0.02)

In [ ]:
# Plot all Be 4f spectra and their components
for uuid in BeTa_initial_state_wcs:
    plot_spectra(uuid, energy_range=[20,26], energy_grid=0.02)

# Be-W

In [ ]:
# Plot all Be 1s spectra and their components
for uuid in bew_init_wc_uuids:
    plot_spectra(uuid, energy_range=[109.5,113.2], energy_grid=0.02)

In [ ]:
# Plot all W 4f spectra and their components
for uuid in bew_init_wc_uuids:
    plot_spectra(uuid, energy_range=[29.0,35.5], energy_grid=0.02)

In [ ]:
#?plot_spectra()

In [ ]:
#3 How would Be2W and Be12W look together?

In [ ]:
be_w = ['4f685bc5-b5fb-46d3-aad6-e0f512c3313d', '045d3071-f442-46b4-8d6b-3c85d72b24d4'] # Be12W Be2W

In [ ]:
# Be 1s

In [ ]:
a,b,c = plot_spectra(be_w, energy_range=[109.5,113.2], energy_grid=0.02)

In [ ]:
a,b,b = plot_spectra(be_w, energy_range=[29.0,35.5], energy_grid=0.02)

In [ ]:
# You can change the ratios:

In [ ]:
a,b,c = plot_spectra(be_w, factors=[1,5], energy_range=[109.5,113.2], energy_grid=0.02)

In [ ]:
a,b,c = plot_spectra(be_w, factors=[1,5], energy_range=[29.0,35.5], energy_grid=0.02)

In [ ]:
be_w_w = ['4f685bc5-b5fb-46d3-aad6-e0f512c3313d', 
          '045d3071-f442-46b4-8d6b-3c85d72b24d4', 
          'f8b12b23-0b71-45a1-9040-b51ccf379439'] # Be12W Be2W, W

In [ ]:
# Be2W stoic, but with Be12W + Be2W + 5W
a,b,c = plot_spectra(be_w_w, factors=[1,0.25,5], energy_range=[29.0,35.5], energy_grid=0.02)

In [ ]:
# Be2W stoic, but with Be12W + 5 Be2W + 5W
a,b,c = plot_spectra(be_w_w, factors=[1,5*0.25,5], energy_range=[29.0,35.5], energy_grid=0.02)

In [ ]:
a,b,c = plot_spectra(be_w_w, factors=[1,1,5], energy_range=[29.0,35.5], energy_grid=0.02)

# Comments/Outlook

To keep Stoichiometry constant we need to be able to add, Elements (corelevel shifts 0)
in order to do this in the current framework, we need to have an initial state workchain run of these elements. (Always good to check anyway)
It might be an Idea to allow the plot spectrum method to deal with fleur_scf_wc nodes, but this case might be missleading, because if one plots a scf of some alloy, their corelevelshifts will be by default also 0.... The current plot method also does not have the option to show only the contributions from the individual alloys as one line. For this at least one interface has to change slightly. But one can currently plot them in seperate plots for sure.

Also this method still needs the logic to deal with the corehole workchain results anyway. I could not test surfaces yet, since I do not have a initial_cls_wc run of a surface yet. Maybe for the surfaces one would like to have additional parameters (penetration depth and spotsize?  to accound for an angular dependence). 